# Snapshots
## Using Snapshots to enable business continuty for your data

Snapshots are zero space copies of what the data on your volume looked like for a particluar point in time.

<p align="center">
  <img src="Pictures/edf-snapshots-overview.png" width="600" height="400" alt="Data Fabric Snapshots">                                                    
  
</p>

### Features of Ezmeral Data Fabric Snapshots:

- Consistent, Point-In-Time image of a volume
- Can be created on-demand or scheduled
- Lightning fast creation, even at 1PB of data
- Stores incremental changes
- Uses Redirect on-write technology, which saves space on changes
- Zero performance loss on writing to original data
- Proven and battle-tested technology in the storage industry


In [ ]:
%login {{ hostvars[inventory_hostname]['IP-WKSHP-Data-Fabric101'] }}

echo "We are now connected to the appliance"

#Variables used by scripts
#password="{{ PASSSTU }}" # your Data Fabric login credentials - password - 
#apiserver_host="{{ DATAFABRICAPISERVERIP }}" # API Server for the Data Fabric Cluster
#ad_group="{{ DATAFABRICADGROUP }}" # LDAP Domain Group for Data Fabric users
#vol_name="{{ DATAFABRICVOL }}" # suffix for student volume name
#snapvol_name="{{ DATAFABRICSNAPVOL }}" # suffix for student snap volume name
#cluster_name="{{ DATAFABRICCLUSTERAME }}" # Data Fabric Cluster Name
#src_mirrvol1="{{ DATAFABRICVOLSRCVOLNAME }}" # suffix for student source mirror volume name
#dst_mirrvol1="{{ DATAFABRICVOLDSTVOLNAME }}"  # suffix for student destination mirror volume name
#secvol_name="{{ DATAFABRICSECVOL }}"  # suffix for student volume name used for security comparison / user access 
#sec_policy="student{{ STDID }}-policy" #security policy to allow r/w access to volumes

echo "your operation context is:" student{{ STDID }} "with Password:" {{ PASSSTU }}

## 1. Creating a snapshot

#### Background:

You have been given a user that is allowed to create and modify volumes on the cluster.  Data Fabric volumes can be mounted so they are accessable from NFS, HDFS or POSIX commands (like the bash kernel shell in these labs).  For security reasons, you can only mount volumes in the /user directory.

First, let's create a volume to work with and create snapshots on:

In [ ]:
source ~/.bashrc

curl -k -X POST 'https://'$apiserver_host':8443/rest/volume/create?name='$username-$snapvol_name'&path=/user/'$username-$snapvol_name'' --user $username:$password

In order to use and demonstrate snapshots effectively we need to write some data to our volume and then verify the data was created.

In [ ]:
source ~/.bashrc
echo "## Creating a file before we take a snapshot.##"
echo "$username-$snapvol_name snapshot volume update $(date)" >> /mapr/$cluster_name/user/$username-$snapvol_name/$username-pre-snapshot-file1
echo "## Listing out our directory to view the file we just created.##"
ls /mapr/$cluster_name/user/$username-$snapvol_name
echo "## Verify the file we just wrote has the correct contents.##"
cat /mapr/$cluster_name/user/$username-$snapvol_name/$username-pre-snapshot-file1

Now that we have created the volume and created data within it we can create an "On-demand" snapshot:

In [ ]:
source ~/.bashrc

curl -k -X POST 'https://'$apiserver_host':8443/rest/volume/snapshot/create?volume='$username-$snapvol_name'&snapshotname='$username-snapshot1'' --user $username:$password

Lastly, let's create another file to showcase the difference in file listing of the snapshot in a future step.

In [ ]:
source ~/.bashrc
echo "## Creating a file after we took a snapshot.##"
echo "$username-$snapvol_name snapshot volume update $(date)" >> /mapr/$cluster_name/user/$username-$snapvol_name/$username-post-snapshot-file1
echo "## Listing out our directory to view the file we just created.##"
ls /mapr/$cluster_name/user/$username-$snapvol_name
echo "## Verify the file we just wrote has the correct contents.##"
cat /mapr/$cluster_name/user/$username-$snapvol_name/$username-post-snapshot-file1

## 2. Listing the snapshot you just created:

In [ ]:
source ~/.bashrc

curl -k -X POST 'https://'$apiserver_host':8443/rest/volume/snapshot/list' --user $username:$password | jq -r | grep -B 12 -A 2 ''$username-$snapvol_name''

## 3. Viewing data within a snapshot

By default all Snapshots, when created, can be viewed by looking within an explicit hidden directory at the top level within the volume.
For example if you try and list the contents of the top level directory you will not see the hidden dirctory

Example:

In [ ]:
source ~/.bashrc

ls -la /mapr/$cluster_name/user/$username-$snapvol_name/

But if we explicitly set the '.snapshot' directory in our listing we will see our snapshot.

Example:

In [ ]:
source ~/.bashrc

ls /mapr/$cluster_name/user/$username-$snapvol_name/.snapshot

The above listing is your manual snapshot you created in step 1.  Now let's look at the contents within this .snapshot directory

Example:


In [ ]:
source ~/.bashrc

ls /mapr/$cluster_name/user/$username-$snapvol_name/.snapshot/$username-snapshot1

In [ ]:
source ~/.bashrc

echo "If this section was successful you should see your test file you created called:"
echo "$username-pre-snapshot-file1"

## 4. Recovery specific data within a snapshot

Now let's delete some data!


In [ ]:
source ~/.bashrc

rm /mapr/$cluster_name/user/$username-$snapvol_name/$username-pre-snapshot-file1

Verify the previous test files we created is removed:

In [ ]:
source ~/.bashrc

ls /mapr/$cluster_name/user/$username-$snapvol_name/

Now let's recovery the specific file from your snapshot, first let's take a look at data within your snapshot.

In [ ]:
source ~/.bashrc

ls /mapr/$cluster_name/user/$username-$snapvol_name/.snapshot/$username-snapshot1

Verify it's the file you want:

In [ ]:
source ~/.bashrc

cat /mapr/$cluster_name/user/$username-$snapvol_name/.snapshot/$username-snapshot1/$username-pre-snapshot-file1

Recover the individual file from your snapshot to your  volume:

In [ ]:
source ~/.bashrc

cp /mapr/$cluster_name/user/$username-$snapvol_name/.snapshot/$username-snapshot1/$username-pre-snapshot-file1 /mapr/$cluster_name/user/$username-$snapvol_name/

Verify your file is on your volume:

In [ ]:
source ~/.bashrc
echo "##List out the files in your volume.##"
ls /mapr/$cluster_name/user/$username-$snapvol_name/
echo "##Verify the contents of your file you recovered.##"
cat /mapr/$cluster_name/user/$username-$snapvol_name/$username-pre-snapshot-file1

## 5. Roll back your data in a volume using snapshot restore

#### Another feature is the abilty to roll your volumes data back to a point-in-time snapshot with Snapshot Restore

With Snapshot Restore you need:

- A currently created volume.
- A volume with a snapshot.
- You must unmount and then mount the volume again

Snapshot Restore is a disributive process and should only be used when you are declaring a diaster or a business continutity event on your data.

First let's unmount our volume:

In [ ]:
source ~/.bashrc

curl -k -X POST 'https://'$apiserver_host':8443/rest/volume/unmount?name='$username-$snapvol_name'' --user $username:$password

Now you can restore your snapshot to a scheudled or on-demand snapshot

In [ ]:
source ~/.bashrc

curl -k -X POST 'https://'$apiserver_host':8443/rest/volume/snapshot/restore?snapshotname='$username-snapshot1'&volume='$username-$snapvol_name'' --user $username:$password

And then re-mount your volume

In [ ]:
source ~/.bashrc

curl -k -X POST 'https://'$apiserver_host':8443/rest/volume/mount?name='$username-$snapvol_name'&path=/user/'$username-$snapvol_name'' --user $username:$password

Now, verify your files are restored to your snapshot restore time:

In [ ]:
source ~/.bashrc
echo "##List out the files in your volume.##"
ls /mapr/$cluster_name/user/$username-$snapvol_name/
echo "##Verify the contents of your file you recovered.##"
cat /mapr/$cluster_name/user/$username-$snapvol_name/$username-pre-snapshot-file1

For more information on the Snapshot restore feature please see [Ezmeral Data Fabric: Snapshot Restore](https://docs.datafabric.hpe.com/62/AdministratorGuide/Snapshots-Restoring-new.html?hl=snapshot%2Crestore)

## 6. Deleting Snapshots 
### Let's cleanup now

 1. Remove volumes
 2. Remove snapshots

In [ ]:
source ~/.bashrc
echo "Deleting Lab2 snapshots"
echo "Listing Student snapshot and extracting right one with associated snapshot id"
snapid=$(curl -k --user $username:$password -X POST 'https://'$apiserver_host':8443/rest/volume/snapshot/list'| jq -r | grep -B 12 -A 2 ''$username-$snapvol_name'' | grep snapshotid | cut -c22-30)

echo "Deleting LAB2 Snapshot using its id"
curl -k --user $username:$password -X POST 'https://'$apiserver_host':8443/rest/volume/snapshot/remove?snapshots='$snapid

echo "Deleting LAB2 student Snapvol volume"
curl -k -X POST 'https://'$apiserver_host':8443/rest/volume/remove?name='$username-$snapvol_name'' --user $username:$password


In [ ]:
%logout 

## Snapshots Summary

In this example we:

1. Created a volume
2. Placed data in that volume
3. Created a snapshot and then showed how to recover individual data from a snapshot back to into the volume
4. We also showed how to "Roll Back" your entire volumes to a particular snapshot point in time by using Snapshot Restore.

# Using Mirrror's for Disaster Recovery
Let's now showcase how to use the Mirror feature on a volume for disater recovery or for developer tasks.

* [Lab 3 - Using Mirror's for Disaster Recovery](3-WKSHP-Mirrors.ipynb)